In [129]:
#!/home/dillon/miniconda3/ens/test/bin/python 
import sqlite3
import pandas as pd
import subprocess
import os 
import PyPDF2
import argparse

class Library:
    def __init__(self, libname) -> None:
        self.con = sqlite3.connect(libname)
        self.cur = self.con.cursor()
        self.cur.execute("""
                    create table if not exists
                    library(
                        Title string Primary key,
                        Authors string Not Null,
                        Date integer not null,
                        unique(Title) 
                    )
                    """)
        self.cur.execute(""" 
                         create table if not exists
                         tags(
                             Title string Primary key,
                             Tag string not null)
                         """)
        self.cur.execute(""" 
                         create table if not exists 
                         quotes(
                             Title Primary key,
                             Quote not null)
                             """
                         )

    def insert_record(self, info_dict):
        insertstring = "insert or replace into library values ( '{0}', '{1}', {2})".format(info_dict["title"],
                                                                                           info_dict["author"],
                                                                                           info_dict["date"])
        self.cur.execute(insertstring)
        self.con.commit()

    def insert_tags(self, title, taglist):
        for i in taglist:
            insert = """
                insert or replace into tags values(
                    {0}, {1}
                )
                """.format(title, i)
        self.con.commit()

    def print_db(self):
        for r in self.cur.execute("select * from library"):
            print(r)

    def delete_record(self, title):
        delstring = "delete from library where Title = '{0}'".format(title)
        self.cur.execute(delstring)
        self.con.commit()

    def close_con(self):
        self.con.close()

class ScanDir:
    def __init__(self, path) -> None:
        self.pdfmetadata = [] 
        self.path = path
        self.scan(self.path)
        
    def scan(self):
        files = os.listdir(self.path)
        for i in files:
            m = i.find(".")
            d = {} 
            if(i[m:] == ".pdf"):
                r = PyPDF2.PdfFileReader(i)
                r = r.getDocumentInfo()
                d["title"] = r.title
                d["author"] = r.author
                date = r['/CreationDate']
                date = date.split('/')
                d['date'] = date[2][0:4]
                d['keywords'] = r['/Keywords']
                self.pdfmetadata.append(d)  
                



